In [ ]:
import cv2
import numpy as np
import pandas as pd
from huggingface_hub import login
from sklearn.model_selection import train_test_split

In [2]:
login()

Get data

In [ ]:
dfs = []

for i in range(0, 43):
    if i < 10:
        temp_df = pd.read_parquet(f"hf://datasets/MichaelP84/manga-colorization-dataset/data/train-0000{i}-of-00043.parquet", 
                     columns=["bw_image", "color_image"],
                     filters=[("title", "==", "Naruto Full Color")])
    else:
        temp_df = pd.read_parquet(f"hf://datasets/MichaelP84/manga-colorization-dataset/data/train-000{i}-of-00043.parquet", 
                     columns=["bw_image", "color_image"],
                     filters=[("title", "==", "Naruto Full Color")])
    dfs.append(temp_df)

df = pd.concat(dfs, ignore_index=True)

In [17]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2474 entries, 0 to 2473
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   bw_image     2474 non-null   object
 1   color_image  2474 non-null   object
dtypes: object(2)
memory usage: 38.8+ KB


,bw_image,color_image
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...


In [18]:
def preprocess_images(df, target_size=(256, 256)):
    """
    Preprocess images stored as bytes in dataframe to target size numpy arrays
    
    Args:
        df: Pandas dataframe with 'bw_image' and 'color_image' columns containing bytes
        target_size: Tuple (height, width) for output images
        
    Returns:
        Tuple of (bw_images, color_images) as numpy arrays
    """
    bw_images = []
    color_images = []
    
    for idx in range(len(df)):
        try:
            # Process black and white image
            bw_bytes = df['bw_image'].iloc[idx]['bytes']
            bw_img = cv2.imdecode(np.frombuffer(bw_bytes, np.uint8), cv2.IMREAD_GRAYSCALE)
            bw_img = cv2.resize(bw_img, target_size[::-1])
            bw_img = np.expand_dims(bw_img, axis=-1)  # (256, 256, 1)
            bw_images.append(bw_img)
            
            # Process color image
            color_bytes = df['color_image'].iloc[idx]['bytes']
            color_img = cv2.imdecode(np.frombuffer(color_bytes, np.uint8), cv2.IMREAD_COLOR)
            color_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            color_img = cv2.resize(color_img, target_size[::-1])  # (256, 256, 3)
            color_images.append(color_img)
            
        except Exception as e:
            print(f"Error processing row {idx}: {str(e)}")
            continue
    
    # Convert to numpy arrays and normalize
    bw_images = np.array(bw_images, dtype=np.float32) / 255.0
    color_images = np.array(color_images, dtype=np.float32) / 255.0
    
    return bw_images, color_images

In [19]:
bw_arrays, color_arrays = preprocess_images(df)

In [21]:
bw_arrays_train, bw_arrays_test, color_arrays_train, color_arrays_test = train_test_split(bw_arrays, color_arrays, test_size=0.05, random_state=42)

In [22]:
print(f"Grayscale images shape: {bw_arrays_train.shape}")  # (N, 256, 256, 1)
print(f"Colored images shape: {color_arrays_train.shape}")    # (N, 256, 256, 3)
print(f"Grayscale images shape: {bw_arrays_test.shape}")  # (N, 256, 256, 1)
print(f"Colored images shape: {color_arrays_test.shape}")    # (N, 256, 256, 3)

Grayscale images shape: (2350, 256, 256, 1)
Colored images shape: (2350, 256, 256, 3)
Grayscale images shape: (124, 256, 256, 1)
Colored images shape: (124, 256, 256, 3)


In [ ]:
np.save("x_train_grayN2.npy", bw_arrays_train)
np.save("x_train_colorN2.npy", color_arrays_train)
np.save("x_test_grayN2.npy", bw_arrays_test)    
np.save("x_test_colorN2.npy", color_arrays_test)